# 机器学习 实验六

**题目：决策树分类器**  
实验条件：给定数据集 Watermelon-train1.csv 和 Watermelon-train2.csv，及其对应的测试集 Watermelon-test1.csv 和 Watermelon-test2.csv  
实验要求：  
1. 基本要求：
   1. 基于 Watermelon-train1 数据集（只有离散属性），构造 ID3 决策树；  
   2. 基于构造的 ID3 决策树，对数据集 Watermelon-test1 进行预测，输出分类精度。
2. 中级要求：
   1. 对数据集 Watermelon-train2，构造 C4.5 或者 CART 决策树，要求可以处理连续型属性；  
   2. 对测试集 Watermelon-test2 进行预测，输出分类精度。
3. 高级要求：使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

# 导入需要的包

In [19]:
import numpy as np
from collections import Counter

feature_list = ["色泽", "根蒂", "敲声", "纹理"]
feature_dict = {"色泽": ["青绿", "乌黑", "浅白"],
                "根蒂": ["蜷缩", "稍蜷", "硬挺"],
                "敲声": ["浊响", "沉闷", "清脆"],
                "纹理": ["清晰", "稍糊", "模糊"]}
label_list = ["否", "是"]

In [99]:
# 提取数据
def load_data1(path):
    x_train = []
    y_train = []
    with open(path,'r') as f1:
        line = f1.readline()
        line = f1.readline()
        while line:
            line = line.rstrip("\r\n").split(',')
            #print(line)
            tmp = []
            for i in range(4):
                tmp.append(feature_dict.get(feature_list[i]).index(line[i+1]))
            y_train.append(label_list.index(line[5]))
            x_train.append(tmp)
            line = f1.readline()
    return np.array(x_train), np.array(y_train)

x_train1, y_train1 = load_data1('Watermelon-train1.csv')
x_test1, y_test1 = load_data1('Watermelon-test1.csv')

In [132]:
for i in range(y_train1.shape[0]):
    print(x_train1[i], y_train1[i])

[0 0 0 0] 1
[1 0 1 0] 1
[1 0 0 0] 1
[0 0 1 0] 1
[2 0 0 0] 1
[0 1 0 0] 1
[1 1 0 1] 1
[1 1 0 0] 1
[1 1 1 1] 0
[0 2 2 0] 0
[2 2 2 2] 0
[2 0 0 2] 0
[0 1 0 1] 0
[2 1 1 1] 0
[2 0 0 2] 0
[0 0 1 1] 0


In [100]:
def load_data2(path):
    x_train = []
    y_train = []
    with open(path,'r') as f1:
        line = f1.readline()
        line = f1.readline()
        while line:
            line = line.rstrip("\r\n").split(',')
            #print(line)
            tmp = []
            for i in range(4):
                tmp.append(feature_dict.get(feature_list[i]).index(line[i+1]))
            tmp.append(float(line[5]))
            y_train.append(label_list.index(line[6]))
            x_train.append(tmp)
            line = f1.readline()
    return np.array(x_train), np.array(y_train)

x_train2, y_train2 = load_data2('Watermelon-train2.csv')
x_test2, y_test2 = load_data2('Watermelon-test2.csv')

# 基本要求

In [106]:
# 计算数据集的信息熵
def entropy(data):
    label = data[:,-1] if data.ndim==2 else data
    label_counts = Counter(label)
    probs = [count / len(label) for count in label_counts.values()]
    ent = -sum(p * np.log2(p) for p in probs if p>0)
    return ent

In [107]:
# 计算某个特征的信息增益
def gain(x, y, feature_index):
    original_ent = entropy(y)
    if feature_index!=4: # 离散属性值
        weighted_ent = 0
        values = set(x[:, feature_index])
        for value in values:
            subset = x[x[:, feature_index] == value]
            weighted_ent += len(subset) / len(y) * entropy(subset)
        return original_ent - weighted_ent
    else: # 连续属性值，排序后二分
        sorted_x = x[np.argsort(x[:, feature_index])]
        thresholds = [(sorted_x[i, feature_index] + sorted_x[i+1, feature_index]) / 2 for i in range(len(x))]
        gains = []
        for threshold in thresholds:
            less_subset = sorted_x[sorted_x[:, feature_index] <= threshold]
            greater_subset = sorted_x[sorted_x[:, feature_index] > threshold]
            less_weight = len(less_subset) / len(y)
            greater_weight = len(greater_subset) / len(y)
            gain = original_ent - (less_weight * entropy(less_subset) + greater_weight * entropy(greater_subset))
            gains.append(gain)
        best_threshold_idx = np.argmax(gains)
        return thresholds[best_threshold_idx], gains[best_threshold_idx]

In [143]:
def id3(x, y, features):
    # 如果所有样本属于同一类别，返回叶节点
    if len(set(y)) == 1:
        return y[0]

    # 如果没有特征可用，返回叶节点，选择样本最多的类
    if len(features) == 0:
        return Counter(y).most_common(1)[0][0]
    
    # 选择信息增益最大的特征
    best_feature = max(features, key=lambda feature: gain(x, y, feature))
    print("best feature = ",best_feature)
    # 构建树
    tree = {best_feature : {}}
    feature_values = set(x[:, best_feature])
    for value in feature_values:
        print("feature",best_feature,"'s value = ",value)
        condition = x[:,best_feature] == value
        subset_idx = [i for i in range(len(condition)) if condition[i]==True]
        x_subset = x[subset_idx]
        y_subset = y[subset_idx]
        print("subset:")
        for i in range(len(subset_idx)):
            print(x_subset[i], y_subset[i])
        remaining_features = [f for f in features if f!=best_feature]
        print("remaining features: ",remaining_features)
        tree[best_feature][value] = id3(x_subset, y_subset, remaining_features)
    print(tree)
    print("----------------------------------")
    return tree

In [62]:
features1 = [0, 1, 2, 3]
features2 = [0, 1, 2, 3, 4]

In [144]:
id3_tree = id3(x_train1, y_train1, features1)

best feature =  3
feature 3 's value =  0
subset:
[0 0 0 0] 1
[1 0 1 0] 1
[1 0 0 0] 1
[0 0 1 0] 1
[2 0 0 0] 1
[0 1 0 0] 1
[1 1 0 0] 1
[0 2 2 0] 0
remaining features:  [0, 1, 2]
best feature =  0
feature 0 's value =  0
subset:
[0 0 0 0] 1
[0 0 1 0] 1
[0 1 0 0] 1
[0 2 2 0] 0
remaining features:  [1, 2]
best feature =  1
feature 1 's value =  0
subset:
[0 0 0 0] 1
[0 0 1 0] 1
remaining features:  [2]
feature 1 's value =  1
subset:
[0 1 0 0] 1
remaining features:  [2]
feature 1 's value =  2
subset:
[0 2 2 0] 0
remaining features:  [2]
{1: {0: 1, 1: 1, 2: 0}}
----------------------------------
feature 0 's value =  1
subset:
[1 0 1 0] 1
[1 0 0 0] 1
[1 1 0 0] 1
remaining features:  [1, 2]
feature 0 's value =  2
subset:
[2 0 0 0] 1
remaining features:  [1, 2]
{0: {0: {1: {0: 1, 1: 1, 2: 0}}, 1: 1, 2: 1}}
----------------------------------
feature 3 's value =  1
subset:
[1 1 0 1] 1
[1 1 1 1] 0
[0 1 0 1] 0
[2 1 1 1] 0
[0 0 1 1] 0
remaining features:  [0, 1, 2]
best feature =  0
feature 0 '

In [165]:
def predict(tree, sample):
    if isinstance(tree, dict): # 如果当前节点是一个dict，表示还需要往下看
        feature_index = list(tree.keys())[0] # 获取当前节点的划分特征索引
        feature_value = sample[feature_index] # 样本在该特征上的取值
        sub_tree = tree[feature_index][feature_value] # 根据取值获取子树
        return predict(sub_tree, sample) # 继续判断下一个节点
    else:
        return tree # 当前节点是叶节点，直接返回预测类别

In [241]:
correct1 = 0
for i in range(len(y_test1)):
    id3_predict = predict(id3_tree, x_test1[i])
    if id3_predict==y_test1[i]:
        correct1 += 1
accuracy1 = correct1 / len(y_test1)
print("ID3 树的预测结果有 %d/%d 个正确，accuracy = %.2f" % (correct1, len(y_test1), accuracy1))

ID3 树的预测结果有 7/10 个正确，accuracy = 0.70


# 中级要求

In [208]:
def gain_ratio(x, y, feature_index, split_values=None):
    original_ent = entropy(y)
    values = set(x[:, feature_index]) if split_values is None else split_values
    intrinsic_val = 0
    weighted_ent = 0
    for value in set(x[:, feature_index]):
        subset = x[x[:, feature_index] == value]
        intrinsic_val -= len(subset) / len(y) * np.log2(len(subset)) / len(y)
    if feature_index!=4:
        for value in values:
            subset = x[x[:, feature_index] == value]
            weighted_ent += len(subset) / len(y) * entropy(subset)
        gain = original_ent - weighted_ent
        gain_ratio = gain / intrinsic_val if intrinsic_val!=0 else 0
    else:
        sorted_x = x[np.argsort(x[:, feature_index])]
        gains = []
        for value in values:
            less_subset = sorted_x[sorted_x[:, feature_index] <= value]
            greater_subset = sorted_x[sorted_x[:, feature_index] > value]
            less_weight = len(less_subset) / len(y)
            greater_weight = len(greater_subset) / len(y)
            gain = original_ent - (less_weight * entropy(less_subset) + greater_weight * entropy(greater_subset))
            gains.append(gain)
        gain_ratio = max(gains) / intrinsic_val if intrinsic_val!=0 else 0
    return gain_ratio

In [201]:
def find_best_split(x, y, features):
    best_feature = None
    best_split_values = None
    best_gain_ratio = -1
    for feature in features:
        unique_values = set(x[:, feature])
        if len(unique_values) == 1: # 跳过只有一个取值的特征
            continue
        if feature!=4: # 离散特征
            cur_gain_ratio = gain_ratio(x, y, feature)
            split_values = None
        else: # 连续特征
            sorted_values = sorted(unique_values)
            split_values = [(sorted_values[i] + sorted_values[i+1]) / 2 for i in range(len(sorted_values)-1)]
            cur_gain_ratio = max((gain_ratio(x, y, feature, split_values=[value]) for value in split_values),
                                 default=0)
        if cur_gain_ratio > best_gain_ratio:
            best_gain_ratio = cur_gain_ratio
            best_feature = feature
            best_split_values = split_values if split_values is not None else None
    return best_feature, best_split_values

In [213]:
def C4_5(x, y, features):
    # 如果所有样本属于同一类别，返回叶节点
    if len(set(y)) == 1:
        return y[0]

    # 如果没有特征可用，返回叶节点，选择样本最多的类
    if len(features) == 0:
        return Counter(y).most_common(1)[0][0]
    
    # 选择信息增益最大的特征
    best_feature, best_split_values = find_best_split(x, y, features)
    if best_feature is None:
        return Counter(y).most_common(1)[0][0]
    print("best feature = ",best_feature)

    # 构建树
    tree = {best_feature: {}}

    if best_split_values is None: # 离散特征
        for value in set(x[:, best_feature]):
            print("feature",best_feature,"'s split value = ",value)
            condition = x[:,best_feature] == value
            subset_idx = [i for i in range(len(condition)) if condition[i]==True]
            x_subset = x[subset_idx]
            y_subset = y[subset_idx]
            print("subset:")
            for i in range(len(subset_idx)):
                print(x_subset[i], y_subset[i])
            remaining_features = [f for f in features if f!=best_feature]
            print("remaining features: ",remaining_features)
            tree[best_feature][value] = C4_5(x_subset, y_subset, remaining_features)
    else: # 连续特征
        for value in best_split_values:
            print("feature",best_feature,"'s split value = ",value)
            condition = x[:,best_feature] <= value
            subset_idx = [i for i in range(len(condition)) if condition[i]==True]
            x_subset = x[subset_idx]
            y_subset = y[subset_idx]
            print("subset:")
            for i in range(len(subset_idx)):
                print(x_subset[i], y_subset[i])
            remaining_features = [f for f in features if f!=best_feature]
            print("remaining features: ",remaining_features)
            tree[best_feature][f'<= {best_split_values[-1]}'] = C4_5(x_subset, y_subset, remaining_features)
        greater_condition = x[:,best_feature] > best_split_values[-1]
        subset_idx = [i for i in range(len(greater_condition)) if greater_condition[i]==True]
        x_subset = x[subset_idx]
        y_subset = y[subset_idx]
        print("subset:")
        for i in range(len(subset_idx)):
            print(x_subset[i], y_subset[i])
        remaining_features = [f for f in features if f!=best_feature]
        print("remaining features: ",remaining_features)
        tree[best_feature][f'> {best_split_values[-1]}'] = C4_5(x_subset, y_subset, remaining_features)

    print(tree)
    print("----------------------------------")
    return tree

In [214]:
C4_5_tree = C4_5(x_train2, y_train2, features2)

best feature =  2
feature 2 's split value =  0.0
subset:
[0.    0.    0.    0.    0.697] 1
[1.    0.    0.    0.    0.634] 1
[2.    0.    0.    0.    0.556] 1
[0.    1.    0.    0.    0.403] 1
[1.    1.    0.    1.    0.481] 1
[1.    1.    0.    0.    0.437] 1
[2.    0.    0.    2.    0.343] 0
[0.    1.    0.    1.    0.639] 0
[1.   1.   0.   0.   0.36] 0
[2.    0.    0.    2.    0.593] 0
remaining features:  [0, 1, 3, 4]
best feature =  1
feature 1 's split value =  0.0
subset:
[0.    0.    0.    0.    0.697] 1
[1.    0.    0.    0.    0.634] 1
[2.    0.    0.    0.    0.556] 1
[2.    0.    0.    2.    0.343] 0
[2.    0.    0.    2.    0.593] 0
remaining features:  [0, 3, 4]
best feature =  3
feature 3 's split value =  0.0
subset:
[0.    0.    0.    0.    0.697] 1
[1.    0.    0.    0.    0.634] 1
[2.    0.    0.    0.    0.556] 1
remaining features:  [0, 4]
feature 3 's split value =  2.0
subset:
[2.    0.    0.    2.    0.343] 0
[2.    0.    0.    2.    0.593] 0
remaining features

In [236]:
def predict_C4_5(tree, sample):
    if isinstance(tree, dict): # 如果当前节点是一个dict，表示还需要往下看
        feature_index = list(tree.keys())[0] # 获取当前节点的划分特征索引
        feature_value = sample[feature_index] # 样本在该特征上的取值
        for condition, subtree in tree[feature_index].items():
            if isinstance(condition, str):
                condition_value = float(condition.split(' ')[1]) if ' ' in condition else float('inf')
                if(condition.startswith('<=') and feature_value <= condition_value) or \
            (condition_value.startswith('>') and feature_value > condition_value):
                    return predict_C4_5(subtree, sample)
            else:
                condition_value = condition
                if feature_value == condition_value:
                    return predict_C4_5(subtree, sample)
    else:
        return tree # 当前节点是叶节点，直接返回预测类别

In [242]:
correct2 = 0
for i in range(len(y_test2)):
    C4_5_predict = predict_C4_5(C4_5_tree, x_test2[i])
    if C4_5_predict==y_test2[i]:
        correct2 += 1
accuracy2 = correct2 / len(y_test2)
print("C4.5 树的预测结果有 %d/%d 个正确，accuracy = %.2f" % (correct2, len(y_test2), accuracy2))

C4.5 树的预测结果有 4/5 个正确，accuracy = 0.80


# 高级要求

分别对基本要求和中级要求里的 ID3 和 C4.5 决策树进行基于深度剪枝的预剪枝，评估在不同最大深度时模型的性能：

In [267]:
def id3_pre_pruning(x, y, features, max_depth = None, depth=0):
    # 如果所有样本属于同一类别，返回叶节点
    if len(set(y)) == 1:
        return y[0]

    # 如果没有特征可用，返回叶节点，选择样本最多的类
    if len(features) == 0:
        return Counter(y).most_common(1)[0][0]
    
    # 达到深度限制，返回样本最多的类
    if max_depth is not None and depth==max_depth:
        return Counter(y).most_common(1)[0][0]
    
    # 选择信息增益最大的特征
    best_feature = max(features, key=lambda feature: gain(x, y, feature))
    # 构建树
    tree = {best_feature : {}}
    feature_values = set(x[:, best_feature])
    for value in feature_values:
        condition = x[:,best_feature] == value
        subset_idx = [i for i in range(len(condition)) if condition[i]==True]
        x_subset = x[subset_idx]
        y_subset = y[subset_idx]
        remaining_features = [f for f in features if f!=best_feature]
        tree[best_feature][value] = id3_pre_pruning(x_subset, y_subset, remaining_features, max_depth, depth+1)
    return tree

In [269]:
for d in range(len(features1)):
    id3_pre_pruning_tree = id3_pre_pruning(x_train1, y_train1, features1, d+1)
    print(id3_pre_pruning_tree)
    correct3 = 0
    for i in range(len(y_test1)):
        id3_pre_pruning_predict = predict(id3_pre_pruning_tree, x_test1[i])
        if id3_pre_pruning_predict==y_test1[i]:
            correct3 += 1
    accuracy3 = correct3 / len(y_test1)
    print("预剪枝 ID3 树在最大深度 %d 下的预测结果有 %d/%d 个正确，accuracy = %.2f\n" % (d+1, correct3, len(y_test1), accuracy3))

{3: {0: 1, 1: 0, 2: 0}}
预剪枝 ID3 树在最大深度 1 下的预测结果有 8/10 个正确，accuracy = 0.80

{3: {0: {0: {0: 1, 1: 1, 2: 1}}, 1: {0: {0: 0, 1: 1, 2: 0}}, 2: 0}}
预剪枝 ID3 树在最大深度 2 下的预测结果有 6/10 个正确，accuracy = 0.60

{3: {0: {0: {0: {1: {0: 1, 1: 1, 2: 0}}, 1: 1, 2: 1}}, 1: {0: {0: 0, 1: {1: {1: 1}}, 2: 0}}, 2: 0}}
预剪枝 ID3 树在最大深度 3 下的预测结果有 6/10 个正确，accuracy = 0.60

{3: {0: {0: {0: {1: {0: 1, 1: 1, 2: 0}}, 1: 1, 2: 1}}, 1: {0: {0: 0, 1: {1: {1: {2: {0: 1, 1: 0}}}}, 2: 0}}, 2: 0}}
预剪枝 ID3 树在最大深度 4 下的预测结果有 7/10 个正确，accuracy = 0.70



In [270]:
def C4_5_pre_pruning(x, y, features, max_depth=None, depth=0):
    # 如果所有样本属于同一类别，返回叶节点
    if len(set(y)) == 1:
        return y[0]

    # 如果没有特征可用，返回叶节点，选择样本最多的类
    if len(features) == 0:
        return Counter(y).most_common(1)[0][0]
    
    # 达到深度限制，返回样本最多的类
    if max_depth is not None and depth==max_depth:
        return Counter(y).most_common(1)[0][0]
    
    # 选择信息增益最大的特征
    best_feature, best_split_values = find_best_split(x, y, features)
    if best_feature is None:
        return Counter(y).most_common(1)[0][0]
    #print("best feature = ",best_feature)

    # 构建树
    tree = {best_feature: {}}

    if best_split_values is None: # 离散特征
        for value in set(x[:, best_feature]):
            condition = x[:,best_feature] == value
            subset_idx = [i for i in range(len(condition)) if condition[i]==True]
            x_subset = x[subset_idx]
            y_subset = y[subset_idx]
            remaining_features = [f for f in features if f!=best_feature]
            tree[best_feature][value] = C4_5_pre_pruning(x_subset, y_subset, remaining_features, max_depth, depth+1)
    else: # 连续特征
        for value in best_split_values:
            condition = x[:,best_feature] <= value
            subset_idx = [i for i in range(len(condition)) if condition[i]==True]
            x_subset = x[subset_idx]
            y_subset = y[subset_idx]
            remaining_features = [f for f in features if f!=best_feature]
            tree[best_feature][f'<= {best_split_values[-1]}'] = C4_5_pre_pruning(x_subset, y_subset, remaining_features, max_depth, depth+1)
        greater_condition = x[:,best_feature] > best_split_values[-1]
        subset_idx = [i for i in range(len(greater_condition)) if greater_condition[i]==True]
        x_subset = x[subset_idx]
        y_subset = y[subset_idx]
        remaining_features = [f for f in features if f!=best_feature]
        tree[best_feature][f'> {best_split_values[-1]}'] = C4_5_pre_pruning(x_subset, y_subset, remaining_features, max_depth, depth+1)

    return tree

In [272]:
for d in range(len(features2)):
    C4_5_pre_pruning_tree = C4_5_pre_pruning(x_train2, y_train2, features2, d+1)
    print(C4_5_pre_pruning_tree)
    correct4 = 0
    for i in range(len(y_test2)):
        C4_5_pre_pruning_predict = predict_C4_5(C4_5_pre_pruning_tree, x_test2[i])
        if C4_5_pre_pruning_predict==y_test2[i]:
            correct4 += 1
    accuracy4 = correct4 / len(y_test2)
    print("预剪枝 C4.5 树在最大深度 %d 下的的预测结果有 %d/%d 个正确，accuracy = %.2f\n" % (d+1, correct4, len(y_test2), accuracy4))

{2: {0.0: 1, 1.0: 0, 2.0: 0}}
预剪枝 C4.5 树在最大深度 1 下的的预测结果有 5/5 个正确，accuracy = 1.00

{2: {0.0: {1: {0.0: 1, 1.0: 1}}, 1.0: {1: {0.0: 1, 1.0: 0}}, 2.0: 0}}
预剪枝 C4.5 树在最大深度 2 下的的预测结果有 5/5 个正确，accuracy = 1.00

{2: {0.0: {1: {0.0: {3: {0.0: 1, 2.0: 0}}, 1.0: {0: {0.0: 1, 1.0: 1}}}}, 1.0: {1: {0.0: {4: {'<= 0.7464999999999999': 1, '> 0.7464999999999999': 1}}, 1.0: 0}}, 2.0: 0}}
预剪枝 C4.5 树在最大深度 3 下的的预测结果有 5/5 个正确，accuracy = 1.00

{2: {0.0: {1: {0.0: {3: {0.0: 1, 2.0: 0}}, 1.0: {0: {0.0: {3: {0.0: 1, 1.0: 0}}, 1.0: {4: {'<= 0.45899999999999996': 1, '> 0.45899999999999996': 1}}}}}}, 1.0: {1: {0.0: {4: {'<= 0.7464999999999999': {3: {0.0: 1, 1.0: 0}}, '> 0.7464999999999999': 1}}, 1.0: 0}}, 2.0: 0}}
预剪枝 C4.5 树在最大深度 4 下的的预测结果有 4/5 个正确，accuracy = 0.80

{2: {0.0: {1: {0.0: {3: {0.0: 1, 2.0: 0}}, 1.0: {0: {0.0: {3: {0.0: 1, 1.0: 0}}, 1.0: {4: {'<= 0.45899999999999996': 1, '> 0.45899999999999996': 1}}}}}}, 1.0: {1: {0.0: {4: {'<= 0.7464999999999999': {3: {0.0: 1, 1.0: 0}}, '> 0.7464999999999999': 1}}, 1.

可以看到，进行预剪枝后，两种模型的性能在最大深度较小时都有不同程度的提升。这是因为决策树考虑的完全是如何提高训练数据分类的正确性，导致模型很有可能过拟合，对噪声数据非常敏感；基于深度的预剪枝可以提前停止树的增长，提高模型的泛化性能。